In [ ]:
import spacy
from spacy.training.example import Example #Müssen später vermutlich was anderes nehmen
from spacy.training.iob_utils import offsets_to_biluo_tags
import random
import os
from got_2NerTrainingData import training_data_got #sehr viele Warnings
#from training_set import training_data_got #Adriana. Weniger Warnings, aber man muss komisch durch iterieren 

# Load the base model
nlp = spacy.blank("en")

# Create and add the NER pipeline component
ner = nlp.add_pipe("ner")

# Define the labels and add them to the NER component
labels = ["PERSON"]
for label in labels:
    ner.add_label(label)

#for train_data2 in training_data_got:
    # Check entity alignment
for text, annotations in training_data_got:
    doc = nlp.make_doc(text)
    #biluo_tags = offsets_to_biluo_tags(doc, annotations["entities"]) # Mit Svens Daten ging es nicht

    # Initialize the optimizer
    optimizer = nlp.initialize()

# Perform the training
for iteration in range(10):  #10 für die kleine Anzahl. Ansonsten könnte man ja so lange iterieren, bis die Fehler eine gewisse Untergrenze erreicht haben
    random.shuffle(training_data_got)
    losses = {}
    for text, annotations in training_data_got:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses, sgd=optimizer)
    #print("Losses:", losses)

# Save the trained NER model
output_dir = os.getcwd()
nlp.to_disk(output_dir)

# Load the saved model
loaded_nlp = spacy.load(output_dir)


In [ ]:
with open("text.txt", "r") as f:
    text = f.read()
# Load the saved model
loaded_nlp = spacy.load(output_dir)

# Test the loaded NER model
doc = loaded_nlp(text)
for ent in doc.ents:
    print(ent.text, ent.label_) #Laut Recherche leer, weil Datensatz zu klein

In [ ]:
import json

# Laden der validierten Daten aus dem JSON-Dateiformat
with open('relationships.json', 'r') as file:
    validation_data = json.load(file)

# Ausgabe des "data" Objekts
print(validation_data)

# Iteriere über die Sätze im Validierungsdatensatz

for data in validation_data:
    sentence = data['sentence']
    entities = data['relationships']
    predicted_relationships=[]
    for entity in entities:
        firstEntity = entity['firstEntity']
        #relationshipType = entity['relationshipType'] # bei ner ja eignetlich unnötig
        secondEntity = entity['secondEntity']
        predicted_relationships.append(firstEntity)
        predicted_relationships.append(secondEntity)
    predicted_relationships = set(predicted_relationships)
    print(predicted_relationships)
    # Verarbeite den Satz mit dem geladenen NER-Modell
    doc = loaded_nlp(sentence)

    # Vergleiche die vorhergesagten Entitäten mit den tatsächlichen Entitäten
    predicted_entities = [(ent.text, ent.label_) for ent in doc.ents] # wird leer sein (war oben schon leer) , benötigen einen größeren Datensatz
    if predicted_entities == entities: # geht vermutlich bei arrays nicht
        print("Entitäten: Vorhersage stimmt überein")
    else:
        print("Entitäten: Vorhersage stimmt nicht überein")

